# Vanilla CF

> Vanilla counterfactual explanation.

In [83]:
# | default_exp methods.vanilla


In [84]:
#| include: false
%load_ext autoreload
%autoreload 2
from ipynb_path import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [85]:
# | export
from __future__ import annotations
from cfnet.import_essentials import *
from cfnet.interfaces import BaseCFExplanationModule, LocalCFExplanationModule
from cfnet.datasets import TabularDataModule
from cfnet.utils import (
    check_cat_info,
    validate_configs,
    binary_cross_entropy,
    cat_normalize,
    grad_update,
)
from functools import partial

In [86]:
# | exporti
def _vanilla_cf(
    x: jnp.DeviceArray,  # `x` shape: (k,), where `k` is the number of features
    pred_fn: Callable[[jnp.DeviceArray], jnp.DeviceArray],  # y = pred_fn(x)
    n_steps: int,
    lr: float,  # learning rate for each `cf` optimization step
    lambda_: float,  #  loss = validity_loss + lambda_params * cost
    cat_arrays: List[List[str]],
    cat_idx: int,
    restarts: int = 0,
    prng_keys: hk.PRNGSequence = hk.PRNGSequence(31)
) -> jnp.DeviceArray:  # return `cf` shape: (k,)
    def loss_fn_1(cf_y: jnp.DeviceArray, y_prime: jnp.DeviceArray):
        return jnp.mean(binary_cross_entropy(y_pred=cf_y, y=y_prime))

    def loss_fn_2(x: jnp.DeviceArray, cf: jnp.DeviceArray):
        return jnp.mean(optax.l2_loss(cf, x))

    def loss_fn(
        cf: jnp.DeviceArray,  # `cf` shape: (k, 1)
        x: jnp.DeviceArray,  # `x` shape: (k, 1)
        pred_fn: Callable[[jnp.DeviceArray], jnp.DeviceArray],
    ):
        y_pred = pred_fn(x)
        y_prime = 1.0 - y_pred
        cf_y = pred_fn(cf)
        return loss_fn_1(cf_y, y_prime) + lambda_ * loss_fn_2(x, cf)

    @partial(jax.jit, static_argnames=["opt"])
    def gen_cf_step(
        x: jnp.DeviceArray, cf: jnp.DeviceArray, opt_state: optax.OptState, opt
    ) -> Tuple[jnp.DeviceArray, optax.OptState]:
        cf_grads = jax.grad(loss_fn)(cf, x, pred_fn)
        cf, opt_state = grad_update(cf_grads, cf, opt_state, opt)
        cf = cat_normalize(cf, cat_arrays=cat_arrays, cat_idx=cat_idx, hard=False)
        cf = jnp.clip(cf, 0.0, 1.0)
        return cf, opt_state

    def gen_cf(x: jnp.DeviceArray, lr: float, n_steps: int, key: random.PRNGKey):
        cf = jnp.array(x, copy=True) \
            + jax.random.normal(key, shape=x.shape, dtype=x.dtype)
        opt = optax.adam(lr)
        opt_state = opt.init(cf)
        for _ in tqdm(range(n_steps)):
            cf, opt_state = gen_cf_step(x, cf, opt_state, opt)
        return cf

    x_size = x.shape
    if len(x_size) > 1 and x_size[0] != 1:
        raise ValueError(
            f"""Invalid Input Shape: Require `x.shape` = (1, k) or (k, ),
but got `x.shape` = {x.shape}. This method expects a single input instance."""
        )
    if len(x_size) == 1:
        x = x.reshape(1, -1)
    
    runs = restarts + 1
    if runs > 1:
        # TODO: use `vmap` for acceleration
        cfs = jnp.concatenate([
            gen_cf(x, lr, n_steps, next(prng_keys)) for _ in range(runs)
        ], axis=0) # (n, k)
        cfs_loss = jnp.array([
            loss_fn(cf.reshape(1, -1), x, pred_fn) for cf in cfs
        ])
        cf = cfs[jnp.argmin(cfs_loss)].reshape(1, -1)
    else:
        cf = gen_cf(x, lr, n_steps, next(prng_keys))

    cf = cat_normalize(cf, cat_arrays=cat_arrays, cat_idx=cat_idx, hard=True)
    return cf.reshape(x_size)

In [87]:
# | export
class VanillaCFConfig(BaseParser):
    n_steps: int = 1000
    lr: float = 0.01
    lambda_: float = 0.01  # loss = validity_loss + lambda_params * cost
    restarts: int = 0 # if `0`, no restarts
    seed: int = 31


In [88]:
# | export
class VanillaCF(LocalCFExplanationModule):
    name = "VanillaCF"

    def __init__(
        self,
        configs: dict | VanillaCFConfig,
        data_module: Optional[TabularDataModule] = None,
    ):
        self.configs = validate_configs(configs, VanillaCFConfig)
        if data_module:
            self.update_cat_info(data_module)

    def generate_cf(
        self,
        x: jnp.ndarray,  # `x` shape: (k,), where `k` is the number of features
        pred_fn: Callable[[jnp.DeviceArray], jnp.DeviceArray],
    ) -> jnp.DeviceArray:
        return _vanilla_cf(
            x=x,  # `x` shape: (k,), where `k` is the number of features
            pred_fn=pred_fn,  # y = pred_fn(x)
            n_steps=self.configs.n_steps,
            lr=self.configs.lr,  # learning rate for each `cf` optimization step
            lambda_=self.configs.lambda_,  #  loss = validity_loss + lambda_params * cost
            cat_arrays=self.cat_arrays,
            cat_idx=self.cat_idx,
            restarts=self.configs.restarts,
            prng_keys=hk.PRNGSequence(self.configs.seed)
        )

    @check_cat_info
    def generate_cfs(
        self,
        X: jnp.DeviceArray,  # `x` shape: (b, k), where `b` is batch size, `k` is the number of features
        pred_fn: Callable[[jnp.DeviceArray], jnp.DeviceArray],
        is_parallel: bool = False,
    ) -> jnp.DeviceArray:
        def _generate_cf(x: jnp.DeviceArray) -> jnp.ndarray:
            return self.generate_cf(x, pred_fn)

        return (
            jax.vmap(_generate_cf)(X) if not is_parallel else jax.pmap(_generate_cf)(X)
        )


## Test

In [40]:
data_configs = {
    "data_dir": "assets/data/s_adult.csv",
    "data_name": "adult",
    "batch_size": 256,
    "sample_frac": 0.1,
    "continous_cols": ["age", "hours_per_week"],
    "discret_cols": [
        "workclass",
        "education",
        "marital_status",
        "occupation",
        "race",
        "gender",
    ],
}
m_configs = {
    "sizes": [50, 10, 50],
    "dropout_rate": 0.3,
    "lr": 0.03,
}
t_configs = {
    "n_epochs": 1,
    "monitor_metrics": "val/val_loss",
    "seed": 42,
    "batch_size": 256,
}


In [41]:
from cfnet.module import PredictiveTrainingModule
from cfnet.train import train_model


In [32]:
training_module = PredictiveTrainingModule(m_configs)

params, opt_state = train_model(
    training_module, TabularDataModule(data_configs), t_configs
)


Epoch 0: 100%|██████████| 10/10 [00:10<00:00,  1.09s/batch, train/train_loss_1=0.0911]
/home/birk/code/cfnet/cfnet/_ckpt_manager.py:14: FutureWarning: jax.tree_leaves is deprecated, and will be removed in a future release. Use jax.tree_util.tree_leaves instead.
  for x in jax.tree_leaves(state):


In [89]:
dm = TabularDataModule(data_configs)
pred_fn = lambda x: training_module.forward(
    params, random.PRNGKey(0), x, is_training=False
)

cf_exp = VanillaCF(configs=VanillaCFConfig(n_steps=1000, restarts=5))
cf_exp.update_cat_info(dm)

X, y = dm.test_dataset[:]

cf = cf_exp.generate_cf(X[0], pred_fn)
cfs = cf_exp.generate_cfs(X[:100, :], pred_fn)


100%|██████████| 1000/1000 [00:05<00:00, 195.44it/s]
